2025-1 생물정보학 및 실습 1 (001)

Your Own Analysis 1

2023-10131 임춘선

In [ ]:
# Google Colab을 이용함

# 가설: ES cell에서 LIN28A의 ER 근처로의 localization은 lncRNA로 인한 것이다.

# Data 출처
# RNALocate (http://www.rnalocate.org/)
# lncATLAS (https://lncatlas.crg.eu/)

In [150]:
import pandas as pd
import numpy as np

In [161]:
# human RNA의 sequence가 담긴 data

human_RNA_seq = pd.read_csv('/content/drive/MyDrive/bioinfo/RNALocate/human_RNA_sequence.txt', sep='\t')

In [162]:
human_RNA_seq

,NCBI_Taxonomy_ID,Species,Gene_ID,Gene_Symbol,Refseq_id,Sequence
0,9606,Homo sapiens,NCBI:1,A1BG,NM_130786.4,ATTGCTGCAGACGCTCACCCCAGACACTCACTGCACCGGAGTGAGC...
1,9606,Homo sapiens,NCBI:10,NAT2,NM_000015.3,ACTTTATTACAGACCTTGGAAGCAAGAGGATTGCATTCAGCCTAGT...
2,9606,Homo sapiens,NCBI:10,NAT2,XM_054359513.1,TTTGGACTAAGGCGGATAATTGGCAAAGGACATTCTCTTCCTGTCC...
3,9606,Homo sapiens,NCBI:10,NAT2,XM_017012938.2,TGAGGCATGTCAGTCACTGCCCCACAACTGGAGCCAGCAACTGGTT...
4,9606,Homo sapiens,NCBI:100,ADA,NR_136160.2,GCTGGCCCCAGGGAAAGCCGAGCGGCCACCGAGCCGGCAGAGACCC...
...,...,...,...,...,...,...
231461,9606,Homo sapiens,miRBase:MIMAT0011163,hsa-miR-548q,NaN,GCUGGUGCAAAAGUAAUGGCGG
231462,9606,Homo sapiens,miRBase:MIMAT0028227,hsa-miR-7158-3p,NaN,CUGAACUAGAGAUUGGGCCCA
231463,9606,Homo sapiens,miRBase:MIMAT0028219,hsa-miR-7154-3p,NaN,AGGAGGACAAGUUGUGGGAU
231464,9606,Homo sapiens,miRBase:MIMAT0018993,hsa-miR-4466,NaN,GGGUGCGGGCCGGCGGGG


In [163]:
# Refseq_id가 NaN인 경우 제거 (microRNA 제거)

human_RNA_seq = human_RNA_seq[ human_RNA_seq['Refseq_id'].notna() ]

In [164]:
# Refseq_id가 NR로 시작하는 경우를 추출 (실험적으로 확인된 non-coding RNA)

human_ncRNA_seq = human_RNA_seq[ human_RNA_seq['Refseq_id'].str.startswith('NR') ]

In [165]:
human_ncRNA_seq

,NCBI_Taxonomy_ID,Species,Gene_ID,Gene_Symbol,Refseq_id,Sequence
4,9606,Homo sapiens,NCBI:100,ADA,NR_136160.2,GCTGGCCCCAGGGAAAGCCGAGCGGCCACCGAGCCGGCAGAGACCC...
43,9606,Homo sapiens,NCBI:100009613,LINC02584,NR_103835.1,AATAGTCACCGGGGATGCAAGCATCAGGGAAGAATACCTCAGAGCA...
44,9606,Homo sapiens,NCBI:100009667,POU5F1P5,NR_131184.1,ATCCAGTCCCAGGACATCTCAAAGCTCTGCAGAAAGAACTTGAGCA...
45,9606,Homo sapiens,NCBI:100009676,ZBTB11-AS1,NR_024407.1,GTGGGAAGTGCGGCTCCTTTCGCGTCCCCCACCCTCTCGGCTCCGC...
72,9606,Homo sapiens,NCBI:100033407,VN2R19P,NR_171678.1,GGCAGTCTGTTGGGCTTTCCTTCCAGTTTGTGGAAGTTTGTTGTTT...
...,...,...,...,...,...,...
227184,9606,Homo sapiens,NCBI:9988,DMTF1,NR_024550.2,ACTCCAGCTGCAGCCACTCTCGCCCGTGGCTGCTTCCTCCATCCTG...
227218,9606,Homo sapiens,NCBI:9989,PPP4R1,NR_052003.2,ACCGGCTCCCCGGCTCGCGCCGCTCCATGTAGCCCCGGCTCCCCGG...
227226,9606,Homo sapiens,NCBI:9989,PPP4R1,NR_168403.1,ACCGGCTCCCCGGCTCGCGCCGCTCCATGTAGCCCCGGCTCCCCGG...
227239,9606,Homo sapiens,NCBI:9989,PPP4R1,NR_168404.1,ACCGGCTCCCCGGCTCGCGCCGCTCCATGTAGCCCCGGCTCCCCGG...


In [166]:
# sequence에서 LIN28A의 binding motif인 UGUG, AAGNG, AAGNNG의 갯수를 세는 함수

# 아직 stem-loop 구조를 고려하지 않음

def count_motif(seq) :

    seq = seq.upper()

    seq += '!!' #찾는 motif의 길이가 4~6이므로 마지막에 2개의 서열을 padding

    cnt = 0

    i = 0

    while i <= len(seq)-6 :

        slide = seq[i:i+4]
        if slide == 'TGTG' : #UGUG motif
            cnt += 1
            i += 4
            continue

        slide = seq[i:i+5]
        if slide[:3] == 'AAG' and slide[4] == 'G' : #AAGNG motif
            cnt += 1
            i += 5
            continue

        slide = seq[i:i+6]
        if slide[:3] == 'AAG' and slide[5] == 'G' : #AAGNNG motif
            cnt += 1
            i += 6
            continue

        i += 1

    return cnt

In [168]:
count_motif_list = [ count_motif(seq) for seq in human_ncRNA_seq['Sequence'] ]

In [170]:
human_ncRNA_seq['motif_cnt'] = count_motif_list

<ipython-input-170-ea8345c3457e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  human_ncRNA_seq['motif_cnt'] = count_motif_list


In [171]:
human_ncRNA_seq

,NCBI_Taxonomy_ID,Species,Gene_ID,Gene_Symbol,Refseq_id,Sequence,motif_cnt
4,9606,Homo sapiens,NCBI:100,ADA,NR_136160.2,GCTGGCCCCAGGGAAAGCCGAGCGGCCACCGAGCCGGCAGAGACCC...,28
43,9606,Homo sapiens,NCBI:100009613,LINC02584,NR_103835.1,AATAGTCACCGGGGATGCAAGCATCAGGGAAGAATACCTCAGAGCA...,12
44,9606,Homo sapiens,NCBI:100009667,POU5F1P5,NR_131184.1,ATCCAGTCCCAGGACATCTCAAAGCTCTGCAGAAAGAACTTGAGCA...,21
45,9606,Homo sapiens,NCBI:100009676,ZBTB11-AS1,NR_024407.1,GTGGGAAGTGCGGCTCCTTTCGCGTCCCCCACCCTCTCGGCTCCGC...,22
72,9606,Homo sapiens,NCBI:100033407,VN2R19P,NR_171678.1,GGCAGTCTGTTGGGCTTTCCTTCCAGTTTGTGGAAGTTTGTTGTTT...,15
...,...,...,...,...,...,...,...
227184,9606,Homo sapiens,NCBI:9988,DMTF1,NR_024550.2,ACTCCAGCTGCAGCCACTCTCGCCCGTGGCTGCTTCCTCCATCCTG...,55
227218,9606,Homo sapiens,NCBI:9989,PPP4R1,NR_052003.2,ACCGGCTCCCCGGCTCGCGCCGCTCCATGTAGCCCCGGCTCCCCGG...,65
227226,9606,Homo sapiens,NCBI:9989,PPP4R1,NR_168403.1,ACCGGCTCCCCGGCTCGCGCCGCTCCATGTAGCCCCGGCTCCCCGG...,66
227239,9606,Homo sapiens,NCBI:9989,PPP4R1,NR_168404.1,ACCGGCTCCCCGGCTCGCGCCGCTCCATGTAGCCCCGGCTCCCCGG...,68


In [ ]:
# Your Own Analysis 1의 주요 분석은 여기까지

In [3]:
# lncRNA의 Subcellular_Localization이 담긴 data

lncRNA_subloc = pd.read_csv('/content/drive/MyDrive/bioinfo/RNALocate/lncRNA subcellular localization information.txt', sep='\t')

In [4]:
lncRNA_subloc

,RNALocate_ID,Species,RNA_Symbol,RNA_Type,Subcellular_Localization,GO_Accession,PubMed_ID,RNALocate_Score
0,RL-E-lnc-0473962,Mus musculus,Rmst,lncRNA,Axon,GO:0030424,26464439.0,0.836757
1,RL-E-lnc-0473963,Mus musculus,Malat1,lncRNA,Axon,GO:0030424,26464439.0,0.836757
2,RL-E-lnc-0473964,Mus musculus,Xist,lncRNA,Axon,GO:0030424,26464439.0,0.836757
3,RL-E-lnc-0473965,Mus musculus,Miat,lncRNA,Axon,GO:0030424,26464439.0,0.836757
4,RL-E-lnc-0473966,Mus musculus,7SL,lncRNA,Axon,GO:0030424,26464439.0,0.836757
...,...,...,...,...,...,...,...,...
464747,RL-E-lnc-0938691,Homo sapiens,TDRKH-AS1,lncRNA,Extracellular vesicle,GO:1903561,37488572.0,0.815591
464748,RL-E-lnc-0938692,Homo sapiens,Loc108349490,lncRNA,Extracellular vesicle,GO:1903561,34499402.0,0.694506
464749,RL-E-lnc-0938699,Homo sapiens,PICSAR,lncRNA,Soma,NaN,26222413.0,0.694506
464750,RL-E-lnc-0938701,Homo sapiens,MALAT1,lncRNA,Speckle periphery,NaN,29133588.0,0.694506


In [9]:
# human lncRNA의 Subcellular_Localization이 담긴 data

human_lncRNA_subloc = lncRNA_subloc[ lncRNA_subloc['Species'] == 'Homo sapiens' ]

In [10]:
human_lncRNA_subloc

,RNALocate_ID,Species,RNA_Symbol,RNA_Type,Subcellular_Localization,GO_Accession,PubMed_ID,RNALocate_Score
3672,RL-E-lnc-0473969,Homo sapiens,nHOTAIRM1,lncRNA,Cell body,GO:0044297,37963881.0,0.694506
3673,RL-E-lnc-0474046,Homo sapiens,CCTT,lncRNA,Centromere,NaN,36332605.0,0.694506
3674,RL-E-lnc-0474047,Homo sapiens,AL162231.2,lncRNA,Chromatin,GO:0000785,NaN,0.328976
3675,RL-E-lnc-0474048,Homo sapiens,CIDECP1,lncRNA,Chromatin,GO:0000785,NaN,0.328976
3676,RL-E-lnc-0474049,Homo sapiens,PI4KAP2,lncRNA,Chromatin,GO:0000785,NaN,0.328976
...,...,...,...,...,...,...,...,...
464747,RL-E-lnc-0938691,Homo sapiens,TDRKH-AS1,lncRNA,Extracellular vesicle,GO:1903561,37488572.0,0.815591
464748,RL-E-lnc-0938692,Homo sapiens,Loc108349490,lncRNA,Extracellular vesicle,GO:1903561,34499402.0,0.694506
464749,RL-E-lnc-0938699,Homo sapiens,PICSAR,lncRNA,Soma,NaN,26222413.0,0.694506
464750,RL-E-lnc-0938701,Homo sapiens,MALAT1,lncRNA,Speckle periphery,NaN,29133588.0,0.694506
